In [ ]:
import pandas as pd
import re
from sklearn.utils import shuffle
import random as rand
import numpy as np
import math
import sklearn.naive_bayes as nB
from sklearn.model_selection import KFold, cross_val_score, cross_validate

In [ ]:
data = pd.read_csv('games_clean.csv')
random = 24
data

In [ ]:
for name in data.columns[1:]:
    print("Before", name + '\n', data[data[name].isna()][name])
    data[name].fillna(data[name].mean(), inplace=True)
    print("After", name + '\n', data[data[name].isna()][name])

In [ ]:
data.isna().sum()

In [ ]:
Labels = data['Metacritic'].unique()
Labels

In [ ]:
from scipy.stats import boxcox

exponential = ['Presence', 'OriginalCost', 'Achievements', 'Storage', 'RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip']
lambdas = {}

for name in exponential:
    boc = boxcox(data.loc[:,name].apply(lambda x: x + 1*10**(-10)))
    data.loc[:,name] = boc[0]
    print(name, boc[1])
    lambdas[name] = boc[1] 

In [ ]:
for name in data.columns:
    print('\033[1m'+name+'\033[0;0m')
    print(data[name].unique())

In [ ]:
'''import matplotlib.pyplot as plt
cols = ['Presence', 'Memory', 'ReleaseDate', 'OriginalCost', 'DiscountedCost', 'Achievements', 'Storage', 'RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip']

for name in cols:
    fig, axes= plt.subplots(1,2, gridspec_kw={'width_ratios': [1, 4]}, figsize=(9,5))
    data.boxplot(column=name,ax=axes[0])
    data.hist(column=name, ax=axes[1])
plt.show()'''

In [ ]:
from sklearn import preprocessing

In [ ]:
encoder = preprocessing.LabelEncoder()
Labels_encoded = encoder.fit_transform(Labels)
Labels_encoded

In [ ]:
data_expected = encoder.fit_transform(data['Metacritic'])
data_input = data.loc[:, data.columns != 'Metacritic']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_input, data_expected, test_size=0.3,random_state=404)

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [ ]:
gnb = GaussianNB()
bnb = BernoulliNB()
mnb = MultinomialNB(alpha=1)

In [ ]:
data['Metacritic'].dtype

In [ ]:
hist = bnb.fit(X_train, y_train)

In [ ]:
y_pred = bnb.predict(X_test)

In [ ]:
encoder.inverse_transform(y_pred)

In [ ]:
from sklearn.metrics import log_loss, accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
log_loss(y_test, y_pred)

In [ ]:
for i in range(20,100):
    print('')
    print('*-----------* KFOLD', i, '*-----------*')
    cv = KFold(n_splits=i, shuffle=False)
    cv_scores = cross_validate(bnb, X=X_train, y=y_train, cv=cv, scoring='accuracy', return_estimator=True)
    print('Max Accuracy in train', '--->', max([(sc,i) for i,sc in enumerate(cv_scores['test_score'])])[0])
    bnb = cv_scores['estimator'][max([(sc,i) for i,sc in enumerate(cv_scores['test_score'])])[1]]
    print('Max Accuracy in test', '--->', accuracy_score(y_test,bnb.predict(X_test)))
    print('')

In [ ]:
for i in range(20,100):
    print('')
    print('*-----------* KFOLD', i, '*-----------*')
    cv = KFold(n_splits=i, shuffle=False)
    cv_scores = cross_validate(bnb, X=data_input, y=data_expected, cv=cv, scoring='accuracy', return_estimator=True)
    print('Max Accuracy in train', '--->', max([(sc,i) for i,sc in enumerate(cv_scores['test_score'])])[0])
    bnb = cv_scores['estimator'][max([(sc,i) for i,sc in enumerate(cv_scores['test_score'])])[1]]
    print('Max Accuracy in test', '--->', accuracy_score(data_expected,bnb.predict(data_input)))
    print('')